In [ ]:
import sys
import os
import pandas as pd
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(parent_dir)

In [ ]:
from analysis_helper import get_analysis_df

(full_df,people_df) = get_analysis_df(parent_dir)
event_type='Indian Premier League'
ipl_stats = full_df[full_df['event_name'] == event_type]

In [ ]:
top_batters = ipl_stats.groupby(['batter_name']).agg(
    runs=('batter_runs', 'sum')
)
top_batters = top_batters.reset_index()
top_batters = top_batters[top_batters['runs'] > 100]
top_batters_names = set(top_batters['batter_name'])


In [ ]:
powerplay_stats = ipl_stats[ipl_stats['over'] <= 6]
powerplay_hitters = powerplay_stats.groupby(['batter_name']).agg(
    runs=('batter_runs', 'sum'),
    balls_faced=('match_id', 'count'),
    innings=('match_id', 'nunique')
)
powerplay_hitters = powerplay_hitters.reset_index()
powerplay_dismissals = powerplay_stats[powerplay_stats['is_wicket'] == 1].groupby(['wicket_player_name']).agg(
    dismissals=('wicket_player_name', 'count')
)
powerplay_dismissals = powerplay_dismissals.reset_index()
powerplay_dismissals.rename(columns={'wicket_player_name': 'batter_name'}, inplace=True)
powerplay_hitters = powerplay_hitters.merge(powerplay_dismissals, on='batter_name', how='left')
powerplay_hitters['dismissals'] = powerplay_hitters['dismissals'].fillna(0)
powerplay_hitters = powerplay_hitters.reset_index()
powerplay_hitters = powerplay_hitters[powerplay_hitters['innings'] > 5]
powerplay_hitters['strike_rate'] = (powerplay_hitters['runs'] / powerplay_hitters['balls_faced']) * 100
powerplay_hitters['average'] = powerplay_hitters['runs'] / (powerplay_hitters['dismissals'].replace(0, 1))
powerplay_hitters['average'] = powerplay_hitters['runs'] / (powerplay_hitters['dismissals'].replace(0, 1))
powerplay_hitters['average_balls_faced'] = powerplay_hitters['balls_faced'] / (powerplay_hitters['innings'].replace(0, 1))
powerplay_hitters['expected_runs'] = powerplay_hitters['average_balls_faced']  * powerplay_hitters['strike_rate'] / 100
del powerplay_hitters['index']


In [ ]:
runs = powerplay_hitters.sort_values(by=['runs'], ascending=False)[:10][['batter_name', 'runs','innings','average']]
runs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create horizontal grouped barplot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 9))
ax = sns.barplot(data=runs, x='runs', y='batter_name', orient='h', palette='muted')

for container in ax.containers:
    ax.bar_label(container)

for i, v in enumerate(runs['runs']):
    print(i, v)
    row = runs.iloc[i].to_dict()
    value = f"runs: {row['runs']}, innings: {row['innings']}, average: {row['average']:.2f}"
    ax.text(800, i, value, ha='center', va='center', 
        fontweight='bold', color='white', fontsize=12)


# Add labels and title
ax.set_xlabel('Runs')
ax.set_ylabel('Batter Name')
plt.title('Runs by Batter in Powerplay')


plt.show()

In [ ]:
powerplay_hitters.sort_values(by=['average'], ascending=False)[:10]

In [ ]:
strike = powerplay_hitters.sort_values(by=['strike_rate'], ascending=False)[:10][['batter_name', 'runs','innings','balls_faced','strike_rate']]
strike

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create horizontal grouped barplot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 9))
ax = sns.barplot(data=strike, x='strike_rate', y='batter_name', orient='h', palette='muted')

for container in ax.containers:
    ax.bar_label(container)

for i, v in enumerate(strike['strike_rate']):
    print(i, v)
    row = strike.iloc[i].to_dict()
    value = f"runs: {row['runs']}, innings: {row['innings']}, balls faced: {row['balls_faced']}, strike rate: {row['strike_rate']:.2f}"
    ax.text(60, i, value, ha='center', va='center', 
        fontweight='bold', color='white', fontsize=12)


# Add labels and title
ax.set_xlabel('Strike Rate')
ax.set_ylabel('Batter Name')
plt.title('Strike Rate by Batter in Powerplay')


plt.show()

In [ ]:
impact = powerplay_hitters.sort_values(by=['expected_runs'], ascending=False)[:10]
impact

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create horizontal grouped barplot
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 9))
ax = sns.barplot(data=impact, x='expected_runs', y='batter_name', orient='h', palette='muted')

for container in ax.containers:
    ax.bar_label(container)

for i, v in enumerate(impact['expected_runs']):
    print(i, v)
    row = impact.iloc[i].to_dict()
    value = f"runs: {row['runs']}, innings: {row['innings']}, avg: {row['average']:.2f}, strike rate: {row['strike_rate']:.2f}"
    ax.text(10, i, value, ha='center', va='center', 
        fontweight='bold', color='white', fontsize=12)


# Add labels and title
ax.set_xlabel('Expected Runs')
ax.set_ylabel('Batter Name')
plt.title('Expected Runs by Batter in Powerplay')


plt.show()

In [ ]:
impact.iloc[0].to_dict()

In [ ]:
powerplay_hitters[powerplay_hitters['batter_name'].isin(['RR Pant', 'KL Rahul'])].style.hide(axis="index")